# KoBERT finetuning

In [ ]:
!pip install ipywidgets  # for vscode
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-8jp6_gay
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-8jp6_gay
     |████████████████████████████████| 132 kB 5.5 MB/s 
     |████████████████████████████████| 344 kB 37.8 MB/s 
     |████████████████████████████████| 47.3 MB 65 kB/s 
     |████████████████████████████████| 4.5 MB 31.8 MB/s 
     |████████████████████████████████| 1.2 MB 34.6 MB/s 
     |████████████████████████████████| 3.8 MB 32.5 MB/s 
     |████████████████████████████████| 67 kB 788 kB/s 
     |████████████████████████████████| 6.5 MB 29.3 MB/s 
     |████████████████████████████████| 596 kB 27.3 MB/s 
     |████████████████████████████████| 895 kB 33.5 MB/s 
     |████████████████████████████████| 79 kB 7.3 MB/s 
     |████████████████████████████████| 8.6 MB 32.7 MB/s 
     |████████████████████████████████| 127 kB 50.1 MB/s 
  Created wheel for kobert: filename=kobe

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm

In [ ]:
from kobert import get_tokenizer
from kobert import get_pytorch_kobert_model

In [ ]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
## CPU
#device = torch.device("cpu")

## GPU
device = torch.device("cuda:0")

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model(cachedir=".cache")

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


# 데이터 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
df_train = pd.read_csv('/content/train.csv')
df_test = pd.read_csv('/content/test.csv')
df_pred = pd.read_csv('/content/pred.csv')

In [ ]:
df_train['label'] = df_train['label'].astype('int')
df_test['label'] = df_test['label'].astype('int')
df_pred['label'] = df_pred['label'].astype('int')

In [ ]:
df_pred

,id,document,label
0,0,풀무원 오너 경영 마감하고 전문경영인 체제로…이효율 신임 총괄CEO 선임,0
1,1,[신년사]김성주 이사장 “국민이 주인인 연금 만들어 가겠다”,0
2,2,"한미연합훈련 '단골' 본험 리처드함, 동해 떠난다",0
3,3,"MSCI, 사회책임경영 지수에서 삼성전자 제외",0
4,4,"글로벌 자산운용사 SSGA, 엄태종 한국 대표 신규 선임",0
...,...,...,...
60987,60987,"최태원 ""개척자 정신"" 허태수 ""생태계 확장"" 조현준 ""민첩한 호랑이""",0
60988,60988,"권준학 농협은행장 ""초혁신 디지털뱅크로 도약하자""",0
60989,60989,"[신년사] 권오갑 현대중공업 회장 ""혁신 통해 가치 창출해야""",0
60990,60990,"결국 원전은 빠졌다···환경부, K택소노미 발표",0


In [ ]:
df_train

,id,document,label
0,0,지배구조 후진적 기업일수록 현금 많이 쥐고 있다,0
1,1,KB 윤종규 “ESG경영은 사회·환경과 동반성장하는 것”,1
2,2,"[ESG 분석]에쓰오일, ‘사전 예방적 녹색 경영’…환경 리스크 철저히 대비",1
3,3,"대한항공, SK에너지와 탄소중립항공유 협력…""ESG 경영 실천""",1
4,4,"삼성 준법위원회, ‘노조ㆍ승계ㆍ시민사회’ 문제 중점적으로 들여다본다",1
...,...,...,...
9914,9914,"SK 계열사들 ESG 경영 확대…""파이낸셜 스토리로 딥체인지""",1
9915,9915,"LGD, 차량용 P-OLED '대한민국 기술대상 대통령상 수상",1
9916,9916,"SK종합화학, 美 플라스틱 재활용 기업과 MOU…""ESG 박차""",1
9917,9917,"“네이버, 미래에셋 자사주 맞교환 대주주 지배력 강화 전략”",1


In [ ]:
df_test

,id,document,label
0,0,[국감 2021] “케이큐브홀딩스..사회적 기업으로 전환”,1
1,1,화장품 용기 재활용 등급 표시 면제? 이들이 바꿨다,1
2,2,2021년 중·화학 업계 키워드…친환경·배터리,1
3,3,[김화진 칼럼] 한진칼과 피터 드러커의 조언,1
4,4,"미래에셋증권, 탁구단을 통한 사회공헌 활동으로 ESG 경영 앞장선다.",1
...,...,...,...
4246,4246,지배구조 개편 재시동 건 현대차...개편안 따라 수혜주 주목,1
4247,4247,"한진그룹, 자체 경영발전 방안 발표…지배구조·투명성 강화",1
4248,4248,"LG그룹, ‘서브원 MRO’ 매각 추진…일감 몰아주기 논란 차단",1
4249,4249,[단독] 기업지배구조원도 신한·우리금융 이사 연임 반대,0


In [ ]:
#df_train.to_csv(s')
#df_test.to_csv('test_30.txt')
df_train.to_csv('train.tsv', sep="\t", index = False)
df_test.to_csv('test.tsv', sep="\t", index = False)
df_pred.to_csv('pred.tsv', sep="\t", index = False)

In [ ]:
aa = pd.read_csv('/content/train.tsv', sep="\t")

In [ ]:
aa[2710:]

,id,document,label
2710,2710,"컨설턴트 대표 영입한 호텔롯데, 신동빈 숙원인 IPO 시동건다",1
2711,2711,"포스코·현대제철, 물류 협력으로 ESG경영 맞손",1
2712,2712,"'바닥' 찍은 현대차그룹주, 계속 질주할까",0
2713,2713,빚내서 빚갚는 롯데그룹...중대기로 선 '뉴 롯데',0
2714,2714,"ESG펀드 수익률 TOP 10 살펴보니, 삼성전자 비중이 1위",1
...,...,...,...
9914,9914,"SK 계열사들 ESG 경영 확대…""파이낸셜 스토리로 딥체인지""",1
9915,9915,"LGD, 차량용 P-OLED '대한민국 기술대상 대통령상 수상",1
9916,9916,"SK종합화학, 美 플라스틱 재활용 기업과 MOU…""ESG 박차""",1
9917,9917,"“네이버, 미래에셋 자사주 맞교환 대주주 지배력 강화 전략”",1


In [ ]:
dataset_train = nlp.data.TSVDataset('/content/train.tsv', field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset("/content/test.tsv", field_indices=[1,2], num_discard_samples=1)
dataset_pred = nlp.data.TSVDataset("/content/pred.tsv", field_indices=[1,2], num_discard_samples=1)

In [ ]:
dataset_train[:5]

[['지배구조 후진적 기업일수록 현금 많이 쥐고 있다', '0'],
 ['KB 윤종규 “ESG경영은 사회·환경과 동반성장하는 것”', '1'],
 ['[ESG 분석]에쓰오일, ‘사전 예방적 녹색 경영’…환경 리스크 철저히 대비', '1'],
 ['"대한항공, SK에너지와 탄소중립항공유 협력…""ESG 경영 실천"""', '1'],
 ['삼성 준법위원회, ‘노조ㆍ승계ㆍ시민사회’ 문제 중점적으로 들여다본다', '1']]

In [ ]:
dataset_test[:5]

[['[국감 2021] “케이큐브홀딩스..사회적 기업으로 전환”', '1'],
 ['화장품 용기 재활용 등급 표시 면제? 이들이 바꿨다', '1'],
 ['2021년 중·화학 업계 키워드…친환경·배터리', '1'],
 ['[김화진 칼럼] 한진칼과 피터 드러커의 조언', '1'],
 ['미래에셋증권, 탁구단을 통한 사회공헌 활동으로 ESG 경영 앞장선다.', '1']]

In [ ]:
dataset_pred[:5]

[['풀무원 오너 경영 마감하고 전문경영인 체제로…이효율 신임 총괄CEO 선임', '0'],
 ['[신년사]김성주 이사장 “국민이 주인인 연금 만들어 가겠다”', '0'],
 ["한미연합훈련 '단골' 본험 리처드함, 동해 떠난다", '0'],
 ['MSCI, 사회책임경영 지수에서 삼성전자 제외', '0'],
 ['글로벌 자산운용사 SSGA, 엄태종 한국 대표 신규 선임', '0']]

In [ ]:
len(dataset_pred)

60992

In [ ]:
len(dataset_train)

9919

In [ ]:
len(dataset_test)

4251

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [ ]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)
data_pred = BERTDataset(dataset_pred, 0, 1, tok, max_len, True, False)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)
pred_dataloader = torch.utils.data.DataLoader(data_pred, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    #print(max_vals)
    #print(max_indices)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
max_vals, max_indices = torch.max(out, 1)
print(max_vals)
print(max_indices.tolist())
train_acc = (max_indices == label).sum().data.cpu().numpy()/max_indices.size()[0]

NameError: ignored

In [ ]:
label_list = label.tolist()
type(label_list)

list

In [ ]:
print(accuracy_score(label_list, max_indices.tolist()))

0.8888888888888888


In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

labels = [1, 0, 0, 1, 1, 1, 0, 1, 1, 1]	# 실제 labels
guesses = [0, 1, 1, 1, 1, 0, 1, 0, 1, 0]	# 에측된 결과

print(accuracy_score(labels, guesses))	# 0.3
print(recall_score(labels, guesses))	# 0.42
print(precision_score(labels, guesses))	# 0.5
print(f1_score(labels, guesses))	# 0.46

0.3
0.42857142857142855
0.5
0.4615384615384615


In [ ]:
for e in range(num_epochs+5):
    train_acc = 0.0
    test_acc = 0.0
    test_pre = 0.0
    test_recall = 0.0
    test_f1score = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        max_vals_test, max_indices_test = torch.max(out, 1)
        test_acc += calc_accuracy(out, label)
        test_pre += precision_score(label.tolist(), max_indices_test.tolist())
        test_recall += recall_score(label.tolist(), max_indices_test.tolist())
        test_f1score += f1_score(label.tolist(), max_indices_test.tolist())
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    print("epoch {} test precision {}".format(e+1, test_pre / (batch_id+1)))
    print("epoch {} test recall {}".format(e+1, test_recall / (batch_id+1)))
    print("epoch {} test f1score {}".format(e+1, test_f1score/ (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


  0%|          | 0/155 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.6867784857749939 train acc 0.546875
epoch 1 train acc 0.8442476318484382


  0%|          | 0/67 [00:00<?, ?it/s]

epoch 1 test acc 0.9052048783858486
epoch 1 test precision 0.9244918998206291
epoch 1 test recall 0.9567089734334709
epoch 1 test f1score 0.9398196326813724


...

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


  0%|          | 0/155 [00:00<?, ?it/s]

epoch 10 batch id 1 loss 0.08977720141410828 train acc 0.984375
epoch 10 train acc 0.9850774449564773


  0%|          | 0/67 [00:00<?, ?it/s]

epoch 10 test acc 0.9014735351022665
epoch 10 test precision 0.9106783717860755
epoch 10 test recall 0.9690924017032889
epoch 10 test f1score 0.9384884064193341


## Recall for Negative

In [ ]:
for e in range(num_epochs+5):
    train_acc = 0.0
    test_acc = 0.0
    test_pre = 0.0
    test_recall = 0.0
    test_f1score = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        max_vals_test, max_indices_test = torch.max(out, 1)
        test_acc += calc_accuracy(out, label)
        test_pre += precision_score(label.tolist(), max_indices_test.tolist())
        test_recall += recall_score(label.tolist(), max_indices_test.tolist(), pos_label = 0)
        test_f1score += f1_score(label.tolist(), max_indices_test.tolist())
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    print("epoch {} test precision {}".format(e+1, test_pre / (batch_id+1)))
    print("epoch {} test recall {}".format(e+1, test_recall / (batch_id+1)))
    print("epoch {} test f1score {}".format(e+1, test_f1score/ (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fd9eee4d950>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
A

  0%|          | 0/155 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.21158047020435333 train acc 0.90625
epoch 1 train acc 0.9228782642089094


  0%|          | 0/67 [00:00<?, ?it/s]

epoch 1 test acc 0.9133671918186844
epoch 1 test precision 0.9534294180540248
epoch 1 test recall 0.8368169562314933
epoch 1 test f1score 0.9435964582659259


...

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


  0%|          | 0/155 [00:00<?, ?it/s]

epoch 10 batch id 1 loss 0.004034403711557388 train acc 1.0
epoch 10 train acc 0.9868919610855095


  0%|          | 0/67 [00:00<?, ?it/s]

epoch 10 test acc 0.8996078634604755
epoch 10 test precision 0.910396749049096
epoch 10 test recall 0.6623568928209427
epoch 10 test f1score 0.9369986374641255


In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    test_pre = 0.0
    test_recall = 0.0
    test_f1score = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        max_vals_test, max_indices_test = torch.max(out, 1)
        test_acc += calc_accuracy(out, label)
        test_pre += precision_score(label.tolist(), max_indices_test.tolist())
        test_recall += recall_score(label.tolist(), max_indices_test.tolist())
        test_f1score += f1_score(label.tolist(), max_indices_test.tolist())
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    print("epoch {} test precision {}".format(e+1, test_pre / (batch_id+1)))
    print("epoch {} test recall {}".format(e+1, test_recall / (batch_id+1)))
    print("epoch {} test f1score {}".format(e+1, test_f1score/ (batch_id+1)))

In [ ]:
token_ids[:,0]

tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2], device='cuda:0')

In [ ]:
id_check = []
pred_result = []

In [ ]:
pred_dataloader

In [ ]:
pred_dataloader = torch.utils.data.DataLoader(data_pred, batch_size=1, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(pred_dataloader), total=len(pred_dataloader)):
  token_ids = token_ids.long().to(device)
  segment_ids = segment_ids.long().to(device)
  valid_length= valid_length
  #label = label.long().to(device)
  out = model(token_ids, valid_length, segment_ids)
  max_vals_test, max_indices_test = torch.max(out, 1)
  
  result_batch = max_indices_test.tolist()
  id_list = segment_ids.tolist()

  pred_result.extend(result_batch)
  id_check.extend(id_list)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


  0%|          | 0/60992 [00:00<?, ?it/s]

In [ ]:
len(pred_result)

60992

In [ ]:
df_pred['label'] = pred_result

In [ ]:
calc_accuracy(out, label)

RuntimeError: ignored

In [ ]:
df_pred.to_csv('/content/pred_with_result_new.csv', encoding = 'utf-8-sig', index = False)